<h1 align="center"><font size="5">Capstone Project - The Battle of Neighborhoods</font></h1>


Introduction/Business Problem

I grew up in Taipei, Taiwan and Taipei is the biggest city in Taiwan in terms of population which is about 2.6 million. Taipei is a city that attract lots of tourists annually, and has many attractions for people visiting Taiwan such as one the tallest building Taipei 101. I had the opportunity to study in the United States and San Diego is one of my favorite city in US. The nick name of San Diego is the finest city in the US, attracting hundreds of thousand people every year. I am wondering if a person is debating which cities he should visit and spend one week there, which cities is his final choice.  